# Linear interpolation for temporal gap filling of ET data due to cloud coverage

In order to fill the temporal data gap in ET calculated using eeMETRIC model from Landsat image, we tested linear interpolation algorithm in Google Earth Engine (GEE). Here data gap is due to cloud coverage. To test the temporal data gap filling algorithm, we started with creating synthetic cloud at random location. After that considering the ET data from before and after image for certain time window based on the time/day where we are filling data gaps are selected to get the linear interpolation values. Interpolated ET and original ET image were than exported in csv format for statistical analysis to evaluate the perfomance of interpolation outside the GEE platform.

In [1]:
#Import necessary modules to use gee python api interface
import ee
ee.Initialize(project='enter your GEE cloud project name')  # Initialize with your GEE project name by replacing " enter your GEE cloud project name" with your project name

In [2]:
# Read  daily ET data calculated from eeMETRIC model for all available dates from GEE repository 
ET = ee.ImageCollection("projects/openet/assets/eemetric/conus/gridmet/landsat/c02")

# Note: Geometry polygon for any location of interest could be defined and here we have tested one location at Nebraska
# Define geometry for synthetic cloud
artificial_cloud = ee.Geometry.Polygon(\
        [[[-96.3384851427627, 41.98461447066241],\
          [-96.3384851427627, 41.89727618376869],\
          [-96.19978275018458, 41.89727618376869],\
          [-96.19978275018458, 41.98461447066241]]], None, False)

area2mask=artificial_cloud

In [3]:
# Define function to masking artifical cloud
def artificial_cloud_image(image):
    filters = image.updateMask(masking)
    return filters
    
# Define function to rename time with short name that is associated with image collections
def addTimeBand(image):
    timeImage = image.metadata('system:time_start').rename('timestamp')
    timeImageMasked = timeImage.updateMask(image.mask().select(0))
    return image.addBands(timeImageMasked)

# linear interpolation function for temporal gap filling
def interpolation(image):
    image = ee.Image(image)
    beforeImages = ee.List(image.get('before'))

    beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
    afterImages = ee.List(image.get('after'))
    afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

    t1 = beforeMosaic.select('timestamp').rename('t1')
    t2 = afterMosaic.select('timestamp').rename('t2')

    t = image.metadata('system:time_start').rename('t')

    timeImage = ee.Image.cat([t1, t2, t])

    timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {\
    't': timeImage.select('t'),\
    't1': timeImage.select('t1'),\
    't2': timeImage.select('t2')})

    interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
    result = image.unmask(interpolated)
    return result.copyProperties(image, ['system:time_start'])

# Function to add feature (lat/long) in feature collection to export data in csv format
def addGeom2Property(feature):
    coordinates =ee.Feature(feature).geometry().coordinates()
    long = coordinates.get(0)
    lat  = coordinates.get(1)
    return feature.set({'long':long,'lat':lat})

In [4]:
# select band "et" which ET data from image collections
et = ET.select('et')

# filter the ET data based on the specific rows and path
filtered = et.filter(ee.Filter.And(\
        ee.Filter.eq('wrs2_path',28),\
        ee.Filter.eq('wrs2_row',31)))
# select the image collection with cloud cover less than 10% (image without clouds)
filtered_control = filtered.filter(\
          ee.Filter.lte('CLOUD_COVER',10))

# select the image collection with cloud cover greater than 10%  (image with clouds)
filtered = filtered.filter(ee.Filter.gt('CLOUD_COVER',10))

# creating image with constant value 1 for the pixels and cliped for synthetic cloud
masking = ee.Image.constant(1).clip(artificial_cloud).mask().Not()

# combine the synthetic cloud with the original image and create the image collections with synthetic cloud
filtered_control1 = filtered_control.map(artificial_cloud_image)

In [5]:
# join two image that has sythetic image with image collection filter with cloud greater than 10%
filtered = filtered.merge(filtered_control1)

# now add timeband to rename time
filtered = filtered.map(addTimeBand)

# we define 60 days time window to pick the image for interpolation and converting time into millis
windows_days = 60
millis = ee.Number(windows_days).multiply(1000*60*60*24)
maxDiffFilter = ee.Filter.maxDifference(**{\
  'difference': millis,\
  'leftField': 'system:time_start',\
  'rightField': 'system:time_start'})
print (maxDiffFilter.getInfo())

{'type': 'Filter.maxDifference', 'difference': 5184000000, 'rightField': 'system:time_start', 'leftField': 'system:time_start'}


In [6]:
# create the left field and right field for filter less than equal to
lessEqFilter = ee.Filter.lessThanOrEquals(**{\
  'leftField': 'system:time_start',\
  'rightField': 'system:time_start'})


In [7]:
# create the left field and right field for filter greater than equal to
greaterEqFilter = ee.Filter.greaterThanOrEquals(**{
  'leftField': 'system:time_start',
  'rightField': 'system:time_start'
})

In [8]:
filter1 = ee.Filter.And(maxDiffFilter, lessEqFilter)

In [9]:
join1 = ee.Join.saveAll(**{\
  'matchesKey': 'after',\
  'ordering': 'system:time_start',\
  'ascending': False})

In [10]:
join1Result = join1.apply(**{
  'primary': filtered,
  'secondary': filtered,
  'condition': filter1
})

In [11]:
filter2 = ee.Filter.And(maxDiffFilter, greaterEqFilter)

In [12]:
join2 = ee.Join.saveAll(**{\
  'matchesKey': 'before',\
  'ordering': 'system:time_start',\
  'ascending': True})

In [13]:
join2Result = join2.apply(**{\
    'primary': join1Result,\
    'secondary': join1Result,\
    'condition': filter2})

In [14]:
interpolatedCol = ee.ImageCollection(join2Result.map(interpolation))

In [15]:
filled= interpolatedCol.filter(\
                      ee.Filter.lte('CLOUD_COVER',10))

print(filled.size().getInfo())


122


In [16]:
#print((filled.first().id().getInfo())+'_filled')

In [17]:
size = ee.ImageCollection(filled).size().getInfo()
filledList = ee.ImageCollection(filled).toList(size)
controlList = ee.ImageCollection(filtered_control).toList(size)
print(size)

122


Exporting filled/interpolated image data in csv format with x,y coordinates and ET value for the co-ordinates

In [18]:
for i in range(0,size):
    if(i == size-1):
        print('kk')
    img = ee.Image(filledList.get(i))
    outName = (img.id().getInfo()) + '_filled'
    filled = img.multiply(0.0001)
    sample  = filled.sample(**{
        'region':artificial_cloud,\
        'scale':30,\
        'geometries':True})
     
    sample = sample.map(addGeom2Property)
    task_config = {
        'fileNamePrefix': outName,
        'fileFormat': 'CSV',
        'selectors': ['lat','long','et'],
        'folder': 'Output_LinearInterpolation'
    }
    task = ee.batch.Export.table.toDrive(sample, outName, **task_config)
    #task.start()

kk


In [19]:
for i in range(0,size):
    img = ee.Image(controlList.get(i))
    outName = (img.id().getInfo()) + '_control'
    control = img.multiply(0.0001)
    sample  = control.sample(**{
        'region':artificial_cloud,\
        'scale':30,\
        'geometries':True})
     
    sample = sample.map(addGeom2Property)
    task_config = {
        'fileNamePrefix': outName,
        'fileFormat': 'CSV',
        'selectors': ['lat','long','et'],
        'folder': 'Output_LinearInterpolation'
    }
    task = ee.batch.Export.table.toDrive(sample, outName, **task_config)
    #task.start()